In [33]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
from mtflearn.io import load_image
from stemplot import imshow, colors_from_lbs
import re

Load images, pts and lbs

In [52]:
import glob
def find_imgs(element):
    dm4_files = glob.glob(f"{element}/*.dm4")
    adf1_files = []
    adf2_files = []
    for file in dm4_files:
        if "ADF1" in file and "ADF2" not in file:
            adf1_files.append(file)
        elif "ADF2" in file and "ADF1" not in file:
            adf2_files.append(file)
    return adf1_files, adf2_files

def find_pts_and_lbs(element):
    folder_path = f"{element} pts and lbs"
    npy_files = glob.glob(f"{folder_path}/*.npy")
    pts_files = []
    lbs_files = []
    for file in npy_files:
        file_name = file.split("\\")[-1]
        if "pts" in file_name and "lbs" not in file_name:
            pts_files.append(file)
        elif "lbs" in file_name and "pts" not in file_name:
            lbs_files.append(file)
    return pts_files, lbs_files

def find_matching_images(lbs_filenames, image_filenames):
    matching_images = []
    for lbs_file in lbs_filenames:
        # 去除路径信息
        file_name = lbs_file.split('\\')[-1]

        # 检查文件名是否以数字或 "E" 开头
        if re.match(r'^[0-9E]', file_name):
            # 提取第一个下划线前的部分
            base_name = file_name.split('_')[0]
        else:
            base_name = file_name

        # 查找 "Ti", "V", "Mn", "Co" 中的一个，并提取其后的数字
        match = re.search(r'(Ti|V|Mn|Co)(\d+)', file_name)
        if match:
            element = match.group(1)
            number = match.group(2)
            pattern = f'{base_name}_{element}{number}'

            # 在 image_filenames 中查找匹配项
            for img_file in image_filenames:
                if pattern in img_file:
                    matching_images.append((lbs_file, img_file))
                    break
                    
def find_matching_images(lbs_filenames,pts_filenames, adf1_filenames,adf2_filenames):
    matching_images = []

    for idx,lbs_file in enumerate(lbs_filenames):
        files = []
        files.append(lbs_file)
        files.append(pts_filenames[idx])
        file_name = lbs_file.split('\\')[-1]
        if re.match(r'^[0-9E]', file_name):
            base_name = file_name.split('_')[0]
        else:
            base_name = ''
        match = re.search(r'(Ti|V|Mn|Co)(\d+)', file_name)
        if match:
            number = match.group(2)
            number = number+'.dm4'
            for idx2,img_file in enumerate(adf1_filenames):
                if base_name and (base_name in img_file and number in img_file):
                    files.append(img_file)
                    files.append(adf2_filenames[idx2])
                    break
                elif not base_name and number in img_file:
                    files.append(img_file)
                    files.append(adf2_filenames[idx2])
                    break
        matching_images.append(files)
    return matching_images

In [55]:
Ti_adf1,Ti_adf2 = find_imgs('Ti')
Ti_pts,Ti_lbs = find_pts_and_lbs('Ti')
Ti_pair_files = find_matching_images(Ti_lbs,Ti_pts,Ti_adf1,Ti_adf2)

In [57]:
V_adf1,V_adf2 = find_imgs('V')
V_pts,V_lbs = find_pts_and_lbs('V')
V_pair_files = find_matching_images(V_lbs,V_pts,V_adf1,V_adf2)

In [58]:
Mn_adf1,Mn_adf2 = find_imgs('Mn')
Mn_pts,Mn_lbs = find_pts_and_lbs('Mn')
Mn_pair_files = find_matching_images(Mn_lbs,Mn_pts,Mn_adf1,Mn_adf2)

In [59]:
Co_adf1,Co_adf2 = find_imgs('Co')
Co_pts,Co_lbs = find_pts_and_lbs('Co')
Co_pair_files = find_matching_images(Co_lbs,Co_pts,Co_adf1,Co_adf2)